<a href="https://colab.research.google.com/github/jane-monga/SKYPER/blob/main/skyper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image

# ----------------------------
# 1) Charger image -> numpy
# ----------------------------
def load_image_as_numpy(path: str) -> np.ndarray:
    """
    Charge une image, la convertit en RGB, puis retourne un tableau numpy uint8 (H, W, 3).
    """
    img = Image.open(path).convert("RGB")
    arr = np.array(img, dtype=np.uint8)
    return arr

# ----------------------------
# 2) Créer un filtre de convolution
# ----------------------------
def get_convolution_filter(
    mode: str = "predefined",
    name: str = "sharpen",
    ksize: int = 3,
    random_kind: str = "float",
    random_range: tuple = (-1.0, 1.0),
    normalize: bool = False
) -> np.ndarray:
    """
    Retourne un kernel (matrice) de convolution.
    - mode="random": kernel aléatoire (int ou float)
    - mode="predefined": kernel prédéfini (blur, sharpen, edge, emboss, gaussian)
    """
    if ksize % 2 == 0:
        raise ValueError("ksize doit être impair (ex: 3, 5, 7).")

    if mode == "random":
        if random_kind == "int":
            low, high = random_range
            # pour int, on interprète random_range comme bornes entières
            k = np.random.randint(int(low), int(high) + 1, size=(ksize, ksize), dtype=np.int32).astype(np.float32)
        else:
            low, high = random_range
            k = np.random.uniform(low, high, size=(ksize, ksize)).astype(np.float32)

        if normalize:
            s = np.sum(k)
            if abs(s) > 1e-8:
                k = k / s
        return k

    # --- Pré-définis (3x3 la plupart) ---
    predefined = {
        "blur": np.array([
            [1, 1, 1],
            [1, 1, 1],
            [1, 1, 1]
        ], dtype=np.float32) / 9.0,

        "sharpen": np.array([
            [0, -1, 0],
            [-1, 5, -1],
            [0, -1, 0]
        ], dtype=np.float32),

        "edge": np.array([
            [-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1]
        ], dtype=np.float32),

        "emboss": np.array([
            [-2, -1, 0],
            [-1,  1, 1],
            [ 0,  1, 2]
        ], dtype=np.float32),

        "gaussian": np.array([
            [1, 2, 1],
            [2, 4, 2],
            [1, 2, 1]
        ], dtype=np.float32) / 16.0
    }

    if name not in predefined:
        raise ValueError(f"Filtre inconnu: {name}. Choix: {list(predefined.keys())}")

    return predefined[name]

# ----------------------------
# 3) Appliquer convolution (sans SciPy, 100% NumPy)
# ----------------------------
def apply_convolution(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """
    Applique une convolution 2D sur une image RGB (H, W, 3).
    Bordure gérée par padding "edge" (répétition du bord).
    Retourne un tableau uint8 (H, W, 3).
    """
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("L'image doit être au format RGB (H, W, 3).")

    kernel = kernel.astype(np.float32)
    kh, kw = kernel.shape
    if kh % 2 == 0 or kw % 2 == 0:
        raise ValueError("Le kernel doit avoir des dimensions impaires.")

    pad_h, pad_w = kh // 2, kw // 2

    # Convertir en float pour calcul
    img = image.astype(np.float32)

    # Padding (edge = répéter les bords)
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode="edge")

    H, W, C = img.shape
    out = np.zeros((H, W, C), dtype=np.float32)

    # Convolution naïve (lisible)
    for y in range(H):
        for x in range(W):
            region = padded[y:y + kh, x:x + kw, :]  # (kh, kw, 3)
            # somme pondérée sur kh,kw pour chaque canal
            out[y, x, :] = np.sum(region * kernel[:, :, None], axis=(0, 1))

    # Clamp [0,255] et retour uint8
    out = np.clip(out, 0, 255).astype(np.uint8)
    return out

# ----------------------------
# 4) Numpy -> image + affichage
# ----------------------------
def numpy_to_image(arr: np.ndarray) -> Image.Image:
    return Image.fromarray(arr, mode="RGB")

def main():
    # --- Chemin image à modifier ---
    path = "image.jpg"  # <-- Mets le chemin de ton image ici

    # 1) Import image
    img_np = load_image_as_numpy(path)

    # 2) Choisir un filtre (prédéfini ou aléatoire)
    # Exemple A: filtre prédéfini
    kernel = get_convolution_filter(mode="predefined", name="sharpen")

    # Exemple B: filtre aléatoire (décommente pour tester)
    # kernel = get_convolution_filter(
    #     mode="random",
    #     ksize=3,
    #     random_kind="float",      # "int" ou "float"
    #     random_range=(-1.0, 1.0),
    #     normalize=False
    # )

    # 3) Appliquer convolution
    filtered_np = apply_convolution(img_np, kernel)

    # 4) Reconversion + affichage
    filtered_img = numpy_to_image(filtered_np)
    filtered_img.show()

    # Optionnel: sauvegarder
    # filtered_img.save("image_filtree.png")

if __name__ == "__main__":
    main()